In [2]:
"""
This notebook is to look at the data and to try to mimic the controller. The reason for this is that the laser does not
accept the commands for some reason when it is on. Must look at the pattern between the controller and the laser and at what intervals
they send each other messages, and which messages they send to each other.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
idle_main_menu = pd.read_csv('idle_main_menu.csv', delimiter = ",")
idle_main_menu

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
...,...,...,...,...
3746,1.757712e+09,0x19000410,3,0a1fff
3747,1.757712e+09,0x19000410,3,021fff
3748,1.757712e+09,0x19000410,3,071fff
3749,1.757712e+09,0x19000410,3,101fff


In [6]:
def device_division(data):
    """
    Enter a dataset and it will return the data divided into the corresponding device.
    "0x18820810" --> The laser
    "0x19000410" --> The controller
    """
    laser = data[data['Arbitration_id'] == '0x18820810']
    controller = data[data['Arbitration_id'] == '0x19000410']
    return laser, controller
    

In [7]:
def different_values(Idle_data, command_data):
    laser_idle = Idle_data[Idle_data['Arbitration_id'] == '0x18820810']
    controller_idle = Idle_data[Idle_data['Arbitration_id'] == '0x19000410']
    
    laser = command_data[command_data['Arbitration_id'] == '0x18820810']
    controller = command_data[command_data['Arbitration_id'] == '0x19000410']

    Lidle_data_unique, Lidle_data_counts = np.unique(laser_idle['data'], return_counts = True)
    Cidle_data_unique, Cidle_data_counts = np.unique(controller_idle['data'], return_counts = True)

    Lcommand_data_unique, Lcommand_data_counts = np.unique(laser['data'], return_counts = True)
    Ccommand_data_unique, Ccommad_data_counts = np.unique(controller['data'], return_counts = True)

    # Now will add the different commands to a list to compare
    similar = []
    different = []

    for message in Ccommand_data_unique:
        if message in Cidle_data_unique:
            similar.append(message)
        else:
            different.append(message)
    # See how many times each one showed
    return similar, different
    

In [8]:
laser_idle, controller_idle = device_division(idle_main_menu)
